In [2]:
import cv2
import numpy as np
import imutils
import os

### Resize the Image

In [3]:
image = cv2.imread("./images/test_2.jpg")
height = image.shape[0]
width = image.shape[1]

ratio = 0.2  # Make it smaller for easy processing
width = int(ratio * width)
height = int(ratio * height)
orig = image.copy()
orig_resized = imutils.resize(image, height=height)
image = imutils.resize(image, height=height)
cv2.imshow("Image",image)
cv2.waitKey(0)


-1

### Turn it to Gray Scale

In [4]:

# Strategy #1
# gray_scaled = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# # Blur to smoothen out and remove unnecessary pixels
# gray_scaled = cv2.GaussianBlur(gray_scaled,(5,5),0)  


#Strategy #2
gray_scaled = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray_scaled = cv2.GaussianBlur(gray_scaled, (7,7), 0)

# dilate helps to remove potential holes between edge segments
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(9,9))
dilated = cv2.morphologyEx(gray_scaled, cv2.MORPH_CLOSE, kernel)


# Find the edges if use, strategy #1 change dilated to gray_scaled
edged_img = cv2.Canny(dilated,0,84)

cv2.imshow("Edges",edged_img)
cv2.waitKey(0)


-1

### Find Countours

In [5]:
#RETR LIST IS BETTER
# Find contours based on OpenCV version
contours, hierarchy = cv2.findContours(edged_img.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
if len(contours) == 3:
    contours = contours[0]  # If OpenCV 3.x, we extract the first element (contours)

# Now grab the contours using imutils
contours = imutils.grab_contours((contours, hierarchy))

# Sort the contours based on size and pick the top 5
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]

# loop over the contours
paper_outline = None
for contour in contours:
    perimeter = cv2.arcLength(contour, True)
    # approximate the contour
    approximation = cv2.approxPolyDP(contour, 0.01 * perimeter, True)

    # if our contour has 4 points, then surely, it should be the paper
    if len(approximation) == 4:
        paper_outline = approximation
        break


### Draw Contours

In [6]:
cv2.drawContours(image,[paper_outline],-1,(255,255,0),2)  
cv2.imshow("Found outline", image)  
cv2.waitKey(0)
    

-1

### Get Corner Points

In [7]:
def order_points(points):
   # initialize a list of co-ordinates that will be ordered
   rectangle = np.zeros((4, 2), dtype="float32")
   sum_points = points.sum(axis=1)
   rectangle[0] = points[np.argmin(sum_points)]  # top-left
   rectangle[2] = points[np.argmax(sum_points)]  # bottom-right
   diff_points = np.diff(points, axis=1)
   rectangle[1] = points[np.argmin(diff_points)]  # top-right
   rectangle[3] = points[np.argmax(diff_points)]  # bottom-left
   return rectangle

### Warp the Image

In [8]:
def set_four_points(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width and height of the new image
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype="float32")

    transform_matrix = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, transform_matrix, (maxWidth, maxHeight))

    return warped

In [9]:
# Resize points to match the original image size
test_points = order_points(paper_outline.reshape(4, 2) * (1 / ratio))

warped = set_four_points(orig, paper_outline.reshape(4, 2) * (1 / ratio))

print("Image Reset in progress")
cv2.imshow("Original", imutils.resize(orig, height))
cv2.imshow("Scanned", imutils.resize(warped, height))
cv2.waitKey(0)

Image Reset in progress


-1

### Enhance the Image

In [10]:

def enhance_image(warped):
    # 1. Adjust contrast
    # We can scale the pixel values to increase contrast.
    contrast_factor = 1.2  # Increase this value to increase contrast further
    enhanced_contrast = cv2.convertScaleAbs(warped, alpha=contrast_factor, beta=0)
    
    # 2. Adjust brightness
    # We can apply a brightness shift (e.g., decrease brightness for better text clarity).
    brightness_factor = -20  # You can adjust this value to lighten or darken the image
    adjusted_brightness = cv2.convertScaleAbs(enhanced_contrast, alpha=1, beta=brightness_factor)
    
    saturation_factor = 1.5
    # 1. Convert the image from BGR to HSV
    hsv = cv2.cvtColor(adjusted_brightness, cv2.COLOR_BGR2HSV)
    
    # 2. Increase saturation by multiplying the saturation channel (index 1) by a factor
    h, s, v = cv2.split(hsv)
    
    # Increase the saturation
    s = cv2.multiply(s, saturation_factor)
    s = np.clip(s, 0, 255)  # Ensure the values stay within the valid range [0, 255]
    
    # 3. Merge the channels back
    hsv_enhanced = cv2.merge([h, s, v])
    
    # 4. Convert back to BGR color space
    enhanced_image = cv2.cvtColor(hsv_enhanced, cv2.COLOR_HSV2BGR)

    return enhanced_image
# Enhance the warped image
enhanced_warped = enhance_image(warped)

# Show the original and enhanced images
cv2.imshow("Original Warped Image", imutils.resize(warped, height))
cv2.imshow("Enhanced Warped Image", imutils.resize(enhanced_warped, height))
cv2.waitKey(0)

-1

### Extract Text

In [11]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
def extract_text(warped_image):
    # 1. Convert the image to grayscale
    gray = cv2.cvtColor(warped_image, cv2.COLOR_BGR2GRAY)
    
    # 2. Apply thresholding to make the text stand out more (you can try different methods like Otsu's thresholding)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 3. Optionally, apply additional preprocessing (e.g., noise reduction or dilation/erosion)
    # For better results, you can use morphological operations like dilation
    kernel = np.ones((1, 1), np.uint8)
    thresh = cv2.dilate(thresh, kernel, iterations=1)  # Optional step, depending on your image

    # 4. Extract text using pytesseract
    text = pytesseract.image_to_string(thresh)

    return text

# Assuming 'warped' is the image you want to extract text from
extracted_text = extract_text(enhanced_warped)

# Print the extracted text
print("Extracted Text:")
print(extracted_text)

Extracted Text:
Mooncake / Mid-Autumn Festival

The moon 1a the fulieat and beightost at the 15th day of the 8th month of the Lunar Cafendar known as thre
Mid-Autumn (Moon) Festival, tt Is also a memorable day for the Chinese ‘who overthrew thier tyrant ruler of

the Yuan Dynasty known as Marce Polo.

Duting thal ero, the people were ail unarmed pnd well guarded by one solder for avery three familles and
they were obliged to maintain all necessities of the soldier.

Due to the Imperietistic tactics of the tyrant the revolutionists tosh advantage of the full moon of
Mid-Autumn. As all tamilina exseinble due te the festival, ney were able to circulate their propaganda
mavement, (o revolt throughout the country thru the messages hidden Inside the mooncakes. Because of
thia the soldiers were unawere snd caught by surprise hy the revolution that succeeded that brought about

the Ning Dynasty in 1387.

MECHANICS:

‘You toll the dice, and you win a prize based on the dice combinations, which 

In [12]:
upload_folder = "./uploads"
if not os.path.exists(upload_folder):
    os.makedirs(upload_folder)

text_output_path = os.path.join(upload_folder,"extracted_text.txt")

# Save the warped (scanned) image
img_output_path = os.path.join(upload_folder, "scanned_image.jpg")
cv2.imwrite(img_output_path, enhanced_warped)
with open(text_output_path, "w") as file:
    file.write(extracted_text)
